# GPT-4-Turbo: The New GPT-4 and What You Need to Know

In [ ]:
OPENAI_API_KEY = ""
EXCHANGE_API = ""

In [ ]:
!pip install openai==1.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
import openai
from openai import OpenAI
import json

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)
client

## Code interpreter

In [ ]:
!wget https://jonfernandes.github.io/files/paris_olympics_1924.csv

--2024-01-10 13:36:42--  https://jonfernandes.github.io/files/paris_olympics_1924.csv
Resolving jonfernandes.github.io (jonfernandes.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jonfernandes.github.io (jonfernandes.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-01-10 13:36:42 ERROR 404: Not Found.



In [ ]:
file = client.files.create(
  file=open("paris_olympics_1924.csv", "rb"),
  purpose='assistants'
)

assistant = client.beta.assistants.create(
  instructions="You are a helpful assistant. Use only the file that has been provided.",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}],
  file_ids=[file.id]
)

print(f"Assistant ID: {assistant.id}")

FileNotFoundError: [Errno 2] No such file or directory: 'paris.csv'

In [ ]:
thread = client.beta.threads.create()

client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What events did eric liddell win a medal in?"
)

thread.id

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

run.id

In [ ]:
import time

def complete_run(thread_id, run_id):
    """Run the assistant on the Thread to get responses"""
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

complete_run(thread.id, run.id)

In [ ]:
def print_messages_from_thread(thread_id):
    """Print messages for thread with thread_id as parameter"""
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

print_messages_from_thread(thread.id)

## Retrieval

In [ ]:
!wget https://jonfernandes.github.io/files/digital-piano.pdf

In [ ]:
file = client.files.create(
  file=open("digital-piano.pdf", "rb"),
  purpose='assistants'
)

assistant = client.beta.assistants.create(
  instructions="You are a helpful assistant. Use only the uploaded document to answer questions.",
  model="gpt-4-1106-preview",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

assistant.id

In [ ]:
thread = client.beta.threads.create()

client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="how do i get the digital piano to play a demo piece?"
)

thread.id

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

run.id

In [ ]:
import time

def complete_run(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

complete_run(thread.id, run.id)

In [ ]:
def print_messages_from_thread(thread_id):
    """Print messages for thread with id: thread_id"""
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

print_messages_from_thread(thread.id)

## Function calling

In [ ]:
prompt = "Convert 50 USD to british pounds"

client.chat.completions.create(
    model="gpt-4-1106-preview",
    temperature=0.0,
    messages=[{"role": "user", "content": prompt}]
)

In [ ]:
def get_details():
    """Do nothing"""
    pass

tools = [
    {
        "type": "function",
        "function": {
            "name" : "get_details",
            "description" : "Convert a given amount of money from one currency to another. Each currency will have a three letter code",
            "parameters": {
                "type": "object",
                "properties": {
                  "base": {
                      "type": "string",
                      "description": "The base or original currency"
                  },
                "target": {
                    "type": "string",
                    "description": "The target or converted currency"
                },
                "amount": {
                    "type": "string",
                    "description": "The amount of money when converting from one currency to another"
                }
                }
            },
            "required": ["base", "target", "amount"]
        }
    }
]

client.chat.completions.create(
      model="gpt-4-1106-preview",
      temperature=0.0,
      messages=[{"role": "user", "content": "Convert 50 USD to british pounds"}],
      tools=tools,
      tool_choice="auto"
)

In [ ]:
def get_chat_completion(prompt, model="gpt-4-1106-preview", tools=None, tool_choice="auto"):
    """Response from model for a given prompt"""
    try:
        completion = client.chat.completions.create(
            model=model,
            temperature=0.0,
            messages=[{"role": "user", "content": prompt}],
            tools=tools,
            tool_choice=tool_choice
        )
    except Exception as e:
        print(e, model, prompt)
    else:
        return completion

response = get_chat_completion("Convert 50 USD to british pounds", tools=tools)
response

In [ ]:
response.choices

In [ ]:
response_arguments = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
response_arguments

In [ ]:
response = get_chat_completion("What's 500 british pounds in Indian currency", tools=tools)

response_arguments = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
response_arguments

In [ ]:
import requests

BASE, TARGET, AMOUNT = response_arguments["base"], response_arguments["target"], response_arguments["amount"]

url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_API}/pair/{BASE}/{TARGET}/{AMOUNT}"
exchange_response = json.loads(requests.get(url).text)
exchange_response

In [ ]:
print(f"{BASE} {AMOUNT} is {TARGET} {exchange_response['conversion_result']}")

In [ ]:
response = get_chat_completion("I woke up a little earlier today", tools=tools)
response

In [ ]:
response.choices[0].message.content

In [ ]:
import requests
import json

def get_chat_completion(prompt, model="gpt-4-1106-preview", tools=None, tool_choice="auto"):
    """Response from model for a given prompt"""
    try:
        completion = client.chat.completions.create(
            model=model,
            temperature=0.0,
            messages=[{"role": "user", "content": prompt}],
            tools=tools,
            tool_choice=tool_choice
        )
    except Exception as e:
        print(e, model, prompt)
    else:
        return completion

def get_details():
    """Do nothing"""
    pass

tools = [
    {
        "type": "function",
        "function": {
            "name" : "get_details",
            "description" : "Convert a given amount of money from one currency to another. Each currency will have a three letter code",
            "parameters": {
                "type": "object",
                "properties": {
                  "base": {
                      "type": "string",
                      "description": "The base or original currency"
                  },
                "target": {
                    "type": "string",
                    "description": "The target or converted currency"
                },
                "amount": {
                    "type": "string",
                    "description": "The amount of money when converting from one currency to another"
                }
                }
            },
            "required": ["base", "target", "amount"],
        }
    }
]


def next_steps(prompt):
    """Determine what needs to be done based on response from the Large Language Model"""
    response = get_chat_completion(prompt, tools=tools)

    if response.choices[0].finish_reason == "stop":
        return f"""Not a currency related question (Did not use tools): {response.choices[0].message.content}"""
    elif response.choices[0].finish_reason == "tool_calls":
        response_arguments = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
        BASE, TARGET, AMOUNT = response_arguments["base"], response_arguments["target"], response_arguments["amount"]
        url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_API}/pair/{BASE}/{TARGET}/{AMOUNT}"
        exchange_response = json.loads(requests.get(url).text)
        return f"{BASE} {AMOUNT} is {TARGET} {exchange_response['conversion_result']}"
    else:
        return NotImplemented

print(next_steps("What's 500 british pounds in whatever they use in Tokyo"))

In [ ]:
print(next_steps("It's sunny today"))

## GPT-4-turbo Vision

In [ ]:
!wget https://github.com/jonfernandes/images/blob/main/JF-bw.png

In [ ]:
!ls -la

In [ ]:
response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://jonfernandes.github.io/images/JF-bw.png",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

response

In [ ]:
response.choices[0].message.content

**Multiple images**

In [ ]:
response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What are in these images? Is there any difference between them?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://jonfernandes.github.io/images/JF-bw.png",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://jonfernandes.github.io/images/JF-color.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0].message.content)

## Context length

In [ ]:
tom_sawyer_text = """

"""

In [ ]:
tom_sawyer_text.split()

In [ ]:
num_tokens = 4 * len(tom_sawyer_text.split()) / 3